<a href="https://colab.research.google.com/github/varshitha775/FMML_LABS_PROJECTS_26/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [11]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [12]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [13]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [14]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [15]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1.Does averaging the validation accuracy across multiple splits give more consistent results?
A)Yes, averaging the validation accuracy across multiple splits generally provides a more consistent and reliable estimate of your model's performance. This approach helps in several ways:

Reduction in Variability: By evaluating your model on multiple validation splits, you account for variability in the data that could affect performance. This helps to smooth out the effects of any one particular data split that might be unusually easy or hard.

Better Generalization Estimate: Averaging across multiple splits provides a better estimate of how your model performs on unseen data. This is because the performance estimate is based on different subsets of the data, which can give a more robust picture of the model's ability to generalize.

Mitigation of Overfitting: A single validation split might not capture the full complexity of the data distribution. By using multiple splits, you reduce the risk of overfitting to a particular split and get a more accurate reflection of how well the model performs on average.

Increased Reliability: A single split can sometimes lead to high variance in performance metrics due to chance. Averaging over multiple splits helps to reduce this variance and provide a more stable measure of performance.

The most common technique for this is k-fold cross-validation, where the data is split into k folds, and the model is trained and validated k times, each time with a different fold as the validation set and the remaining folds as the training set. The validation accuracy is then averaged across these k folds to get a more reliable estimate of performance.

So, using multiple splits and averaging the results is a standard practice to ensure that your performance metrics are more consistent and reflective of your model's true capabilities.




2)Does it give more accurate estimate of test accuracy?
A)Averaging validation accuracy across multiple splits (as in k-fold cross-validation) provides a more reliable estimate of your model's performance on unseen data, but it is not the same as directly estimating test accuracy. Here's a breakdown:

Validation Accuracy vs. Test Accuracy
Validation Accuracy: This is the performance metric you get from evaluating your model on a validation set during the training process. When using k-fold cross-validation, you get an average validation accuracy across different folds. This helps you understand how well your model is performing across different subsets of your training data.

Test Accuracy: This is the performance metric you get from evaluating your final model on a separate, held-out test set that was not used during training or validation. The test set provides a measure of how well your model generalizes to truly unseen data.

Relationship Between Validation and Test Accuracy
Indirect Estimate: The validation accuracy averaged across multiple splits provides an estimate of how well the model is expected to perform on unseen data, but it is still indirectly related to test accuracy. Since the validation set used in cross-validation is part of the training process, it might not fully capture the performance on a completely independent test set.

Generalization Estimate: The averaged validation accuracy gives a good indication of how your model is likely to perform in general, which can be a good proxy for test accuracy. However, it's not a direct measure of test accuracy, which could be slightly different due to the test set being completely independent.

Final Test Evaluation: To obtain the true test accuracy, you should evaluate your final model on a separate test set that was not used in the training or validation process. This is the definitive measure of how well your model generalizes to completely unseen data.

In summary, while averaging validation accuracy across multiple splits provides a reliable estimate of your model’s performance and generalization ability, it is still important to evaluate the final model on an independent test set to get the most accurate measure of test accuracy.




3)What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
A)When referring to "iterations" in the context of model evaluation, it could mean different things depending on the specific aspect being discussed. However, I'll address the concept of iterations in the context of k-fold cross-validation, which is a common method for estimating model performance:

Effect of Number of Folds in Cross-Validation
In k-fold cross-validation, the number of "iterations" refers to the number of folds (k) used. Each iteration involves training and validating the model on different subsets of the data. The effect of the number of folds on the estimate of model performance is as follows:

Higher Number of Folds (e.g., 10-fold vs. 5-fold):

Less Bias: With more folds (e.g., 10-fold), each fold is smaller, which means that the training data in each fold is closer to the full dataset size. This helps in reducing bias in the estimate because the model is validated on data that is closer to the overall data distribution.
More Variability in Training: Each training set is slightly smaller, which can introduce more variability in the performance metrics if the dataset is small.
More Computational Cost: More folds mean more iterations of training and evaluation, which increases computational time and resources.
Lower Number of Folds (e.g., 5-fold vs. 10-fold):

Higher Bias: Fewer folds mean that each training set is larger, which can reduce the variance of the estimate but might introduce bias because each validation set is larger and less representative of the full dataset.
Less Computational Cost: Fewer folds require fewer training iterations, which can reduce the overall computational cost and time.
Impact on Estimate Quality
Accuracy of Estimate: As the number of folds increases, the estimate of model performance becomes more stable and less sensitive to the particular choice of validation subsets. With more folds, each subset of data used for validation is a smaller portion of the entire dataset, leading to a performance estimate that is more representative of the model's behavior across the whole dataset.

Trade-Off: There is a trade-off between computational efficiency and the accuracy of the estimate. While increasing the number of folds can lead to a more accurate estimate, it also increases computational demands. For many practical purposes, 5 or 10 folds are commonly used, as they balance the need for a reliable estimate with computational efficiency.

In summary, increasing the number of iterations (folds) in cross-validation typically provides a more accurate and stable estimate of model performance. However, this comes at the cost of increased computational effort. For most applications, using around 5 to 10 folds is a good balance between accuracy and efficiency.




4)Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
A)Increasing the number of iterations (or folds) in cross-validation does not directly address issues that arise from having a very small training or validation dataset. Here’s why:

Small Dataset Challenges
High Variability: With a very small dataset, each fold in cross-validation will also be small, which can lead to high variability in performance estimates. Each fold might not be representative of the overall dataset, leading to unstable and unreliable performance metrics.

Overfitting Risk: Small training datasets can lead to overfitting because the model may learn noise and specificities of the small dataset rather than general patterns. This issue is not directly mitigated by increasing the number of folds.

Validation Set Representation: When the dataset is very small, even a single fold used for validation might not be representative of the data distribution, regardless of how many folds are used. Increasing the number of folds does not resolve the problem of having an insufficient amount of data for effective validation.

Alternative Approaches for Small Datasets
Instead of simply increasing the number of folds, consider these approaches to deal with small datasets:

Leave-One-Out Cross-Validation (LOOCV):

In LOOCV, each data point is used as a single validation set while the rest of the data points are used for training. This method is especially useful when the dataset is extremely small because it maximizes the amount of data used for training in each iteration. However, it can be computationally expensive and might still suffer from high variability if the dataset is very small.
Data Augmentation:

If applicable, augment your data to artificially increase the size of the dataset. Techniques such as adding noise, rotating, or scaling can help in generating more diverse examples from the existing data.
Regularization:

Apply regularization techniques to reduce the risk of overfitting when training on small datasets. Regularization methods such as L1 or L2 regularization help in constraining the model and improving generalization.
Model Selection:

Choose simpler models or models that are less likely to overfit on small datasets. Complex models with many parameters might not generalize well when trained on limited data.
Bootstrapping:

Use resampling techniques like bootstrapping to generate multiple datasets from the original small dataset. This can provide additional variance estimates and help in assessing the stability of the model’s performance.
Transfer Learning:

If possible, leverage pre-trained models on similar tasks or datasets. Transfer learning can help you build on existing knowledge and improve performance even with limited data.
Summary
Increasing the number of folds in cross-validation can help provide a more stable estimate of performance by averaging over more splits, but it does not address the underlying issue of having a very small dataset. For very small datasets, alternative strategies such as LOOCV, data augmentation, and regularization are more effective in mitigating the challenges associated with limited data

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.